# Bag of words

In [1]:
vocab = {} # maps word to integer representing it
word_encoding = 1
def bag_of_words(text):
    global word_encoding
    
    words = text.lower().split(" ") # create a list of all of the words in the text
    bag = {}  # stores all of the encodings and their frequency
    
    for word in words:
        if word in vocab:
            encoding = vocab[word]  # get encoding from vocab
        else:
            vocab[word] = word_encoding
            encoding = word_encoding
            word_encoding += 1
            
        if encoding in bag:
            bag[encoding] += 1
            
        else:
            bag[encoding] = 1
    return bag

text = 'This is a test to see if this test will work'
bag = bag_of_words(text)
print(bag)
print(vocab)


{1: 2, 2: 1, 3: 1, 4: 2, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1}
{'this': 1, 'is': 2, 'a': 3, 'test': 4, 'to': 5, 'see': 6, 'if': 7, 'will': 8, 'work': 9}


# Sentiment Analysis

Movie Review System

In [14]:
from keras.datasets import imdb # loading IMDB movie review datasets from keras
# This dataset contains revies from IMDB and already proprocessed having label as either positive or negative
# The word is encoded by the integer 3 meaning that it is the third most common word in the dataset

In [15]:
from keras.preprocessing import sequence
import tensorflow as tf
from tensorflow import keras

In [16]:
import os
import numpy as np

In [17]:
VOCAB_SIZE = 88584

MAXLEN = 250           # vocabulary size
BATCH_SIZE = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = VOCAB_SIZE)

In [6]:
# taking a look at a review
train_data[0]

[1,
 14,
 22,
 16,
 43,
 530,
 973,
 1622,
 1385,
 65,
 458,
 4468,
 66,
 3941,
 4,
 173,
 36,
 256,
 5,
 25,
 100,
 43,
 838,
 112,
 50,
 670,
 22665,
 9,
 35,
 480,
 284,
 5,
 150,
 4,
 172,
 112,
 167,
 21631,
 336,
 385,
 39,
 4,
 172,
 4536,
 1111,
 17,
 546,
 38,
 13,
 447,
 4,
 192,
 50,
 16,
 6,
 147,
 2025,
 19,
 14,
 22,
 4,
 1920,
 4613,
 469,
 4,
 22,
 71,
 87,
 12,
 16,
 43,
 530,
 38,
 76,
 15,
 13,
 1247,
 4,
 22,
 17,
 515,
 17,
 12,
 16,
 626,
 18,
 19193,
 5,
 62,
 386,
 12,
 8,
 316,
 8,
 106,
 5,
 4,
 2223,
 5244,
 16,
 480,
 66,
 3785,
 33,
 4,
 130,
 12,
 16,
 38,
 619,
 5,
 25,
 124,
 51,
 36,
 135,
 48,
 25,
 1415,
 33,
 6,
 22,
 12,
 215,
 28,
 77,
 52,
 5,
 14,
 407,
 16,
 82,
 10311,
 8,
 4,
 107,
 117,
 5952,
 15,
 256,
 4,
 31050,
 7,
 3766,
 5,
 723,
 36,
 71,
 43,
 530,
 476,
 26,
 400,
 317,
 46,
 7,
 4,
 12118,
 1029,
 13,
 104,
 88,
 4,
 381,
 15,
 297,
 98,
 32,
 2071,
 56,
 26,
 141,
 6,
 194,
 7486,
 18,
 4,
 226,
 22,
 21,
 134,
 476,
 26,
 480,
 5

More preprocessing

In [7]:
# making each review the same length as we can not pass different length data into our neural network.

train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)

In [8]:
train_data[1] # here bunch of 0's are added to make the length 250( padding from left)

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     1,   194,
        1153,   194,  8255,    78,   228,     5,     6,  1463,  4369,
        5012,   134,    26,     4,   715,     8,   118,  1634,    14,
         394,    20,    13,   119,   954,   189,   102,     5,   207,
         110,  3103,    21,    14,    69,   188,     8,    30,    23,
           7,     4,   249,   126,    93,     4,   114,     9,  2300,
        1523,     5,   647,     4,   116,     9,    35,  8163,     4,
         229,     9,   340,  1322,     4,   118,     9,     4,   130,
        4901,    19,

Creating the Model

In [9]:
# we will use a word embedding layer a the first layer in our model and add a LSTM layer afterwards that feeds into a dense
# node to get predicted sentiment

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),                     # 32 is the output dimension of the vectors generated by the embedding layer
    tf.keras.layers.Dense(1, activation='sigmoid') # sigmoid function squeezes the values between 0 and 1
])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 32)          2834688   
_________________________________________________________________
lstm (LSTM)                  (None, 32)                8320      
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 2,843,041
Trainable params: 2,843,041
Non-trainable params: 0
_________________________________________________________________


Training

In [11]:
# compile and train the model

model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['acc'])
# loss='binary_crossentropy' gives correct loss in the problem

history = model.fit(train_data, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 35s 55ms/step - loss: 0.4281 - acc: 0.8057 - val_loss: 0.4191 - val_acc: 0.8296
Epoch 2/10
625/625 [==============================] - 35s 55ms/step - loss: 0.2437 - acc: 0.9079 - val_loss: 0.2833 - val_acc: 0.8954
Epoch 3/10
625/625 [==============================] - 31s 50ms/step - loss: 0.1883 - acc: 0.9324 - val_loss: 0.3206 - val_acc: 0.8718
Epoch 4/10
625/625 [==============================] - 33s 53ms/step - loss: 0.1533 - acc: 0.9470 - val_loss: 0.3668 - val_acc: 0.8684
Epoch 5/10
625/625 [==============================] - 32s 51ms/step - loss: 0.1272 - acc: 0.9559 - val_loss: 0.3405 - val_acc: 0.8894
Epoch 6/10
625/625 [==============================] - 40s 65ms/step - loss: 0.1121 - acc: 0.9606 - val_loss: 0.2929 - val_acc: 0.8926
Epoch 7/10
625/625 [==============================] - 32s 52ms/step - loss: 0.1000 - acc: 0.9661 - val_loss: 0.3447 - val_acc: 0.8780
Epoch 8/10
625/625 [==============================] - 32s 51ms

Evaluating the model

In [12]:
results = model.evaluate(test_data, test_labels)
print(results)

782/782 [==============================] - 11s 14ms/step - loss: 0.4281 - acc: 0.8571
[0.4280807673931122, 0.8570799827575684]


Making predictions

In [18]:
word_index = imdb.get_word_index()

def encode_text(text):
    tokens = keras.preprocessing.text.text_to_word_sequence(text)
    tokens = [word_index[word] if word in word_index else 0 for word in tokens]
    return sequence.pad_sequences([tokens], MAXLEN)[0]

text = 'That movie was not amazimg as we expected.'
encoded = encode_text(text)
print(encoded)

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0  12  17  13  21   

In [19]:
# make a decode function

reverse_word_index = {value:key for (key, value) in word_index.items()}

def decode_integers(integers):
    PAD = 0
    text = ""
    for num in integers:
        if num != PAD:
            text += reverse_word_index[num] + " "
            
    return text[:-1]
print(decode_integers(encoded))

that movie was not as we expected


In [25]:
# making prediction

def predict(text):
    encoded_text = encode_text(text)
    pred = np.zeros((1,250))
    pred[0] = encoded_text
    result = model.predict(pred)
    print(result[0])
    
positive_review = 'That movie was just amazing! I loved it.'
predict(positive_review)

negative_review = 'That movie really sucked. I absolutely hated it.'
predict(negative_review)

[0.72630507]
[0.36465204]
